## OLP_LLM: Pipeline for generating FOON graphs via LLM Prompting

## Step 1 :- Generate a FOON via LLM Prompting

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### Initialize library

In [1]:
%reload_ext autoreload
%autoreload 2

from olp_library import *

### Language to  OLP

In [2]:
models = [
    "gpt-3.5-turbo",
    "gpt-4",
    ]

in_context_file = "incontext_examples.txt"
user_task = "How do I make coffee?"
user_task = "How do I make a Bloody Mary cocktail?"
# user_task = "How do I make a Greek salad?"

# user_task = "How do I make a burger?"
# user_task = "How do I unlock a door?"
# user_task = "How do I turn on the lights in my house"

high_level_plan, object_level_plan, plan_objects = generate_olp(user_task, in_context_file, models[1], verbose=True)

print(plan_objects)

*************************************************************************
Stage 1 Prompting
*************************************************************************
Model: gpt-4
Complete prompt:
[
    {
        "role": "system",
        "content": "You are an LLM that understands how to generate concise high level plans for arbitrary tasks involving objects and manipulation. Only focus on what happens to objects. Stay consistent with object names and use one verb per step. Assume all objects needed to complete the plan are available: you do not need to retrieve or clean objects."
    },
    {
        "role": "user",
        "content": "How do I make a Bloody Mary cocktail?\nAfter listing the high level plan, list all the unique objects used in the high level plan, including the object created after solving the task, ignoring state changes to those objects. Follow the format 'unique_objects:object_1, object_2, object_3, ...'"
    }
]
****************************************************

In [3]:
## Visualize Outputs
print("***********************\n   High level plan\n***********************\n",   high_level_plan,   "\n")
print("***********************\n   Object level plan\n***********************")
print(*object_level_plan, sep="\n")
print("\n***********************\n   Plan objects\n***********************\n",      plan_objects,      "\n")

***********************
   High level plan
***********************
 1. Place ice cubes into the glass.
2. Pour vodka over the ice in the glass.
3. Add tomato juice to the glass.
4. Squeeze lemon into the glass.
5. Add Worcestershire sauce to the glass.
6. Add Tabasco sauce to the glass.
7. Sprinkle salt and pepper into the glass.
8. Stir the contents of the glass with a stirrer.
9. Garnish the glass with a celery stalk.

unique_objects: glass, ice cubes, vodka, tomato juice, lemon, Worcestershire sauce, Tabasco sauce, salt, pepper, stirrer, celery stalk. 

***********************
   Object level plan
***********************
{'Step': 'Place ice cubes into the glass', 'Objects': ['ice cubes', 'glass'], 'Action': 'Place', 'StateChanges': {'ice cubes': {'Precondition': ['separate'], 'Effect': ['in glass']}, 'glass': {'Precondition': ['empty'], 'Effect': ['contains ice cubes']}}}
{'Step': 'Pour vodka over the ice in the glass', 'Objects': ['vodka', 'glass'], 'Action': 'Pour', 'StateChanges'

In [4]:
## Test Foon creation
plan_step = object_level_plan[0]

sample_unit = create_olp_functionalUnit(plan_step,plan_objects)
sample_unit.print_functions[2](version=1)

Creating functional unit for: Place ice cubes into the glass
Objects: ['ice cubes', 'glass']
Current object is : ice cubes
	 ice cubes state changes: {'Precondition': ['separate'], 'Effect': ['in glass']}
		 Precondition : separate || related objects: None
		 Effect : in glass || related objects: glass
Current object is : glass
	 glass state changes: {'Precondition': ['empty'], 'Effect': ['contains ice cubes']}
		 Precondition : empty || related objects: None
		 Effect : contains ice cubes || related objects: None
O	ice cubes
S	separate
O	glass
S	empty
M	Place	<Assumed>
O	ice cubes
S	in	 [glass]
O	glass
S	contains	{ice cubes}


In [5]:
#Complete FOON prototype:
FOON_prototype = []

for step in object_level_plan:
    print(step['Step'])
    # -- now we will create functional units that follow the FOON format:
    new_unit = create_olp_functionalUnit(step, plan_objects)      
    new_unit.print_functions[2]()

    # -- set output objects as goal nodes:    
    if object_level_plan.index(step) == (len(object_level_plan) - 1):
        for N in range(new_unit.getNumberOfOutputs()):
            new_unit.getOutputNodes()[N].setAsGoal()

    # input()
    print()

    # -- add the functional unit to the FOON prototype:
    FOON_prototype.append(new_unit)

Place ice cubes into the glass
Creating functional unit for: Place ice cubes into the glass
Objects: ['ice cubes', 'glass']
Current object is : ice cubes
	 ice cubes state changes: {'Precondition': ['separate'], 'Effect': ['in glass']}
		 Precondition : separate || related objects: None
		 Effect : in glass || related objects: glass
Current object is : glass
	 glass state changes: {'Precondition': ['empty'], 'Effect': ['contains ice cubes']}
		 Precondition : empty || related objects: None
		 Effect : contains ice cubes || related objects: None
O	ice cubes
S	separate
O	glass
S	empty
M	Place	<Assumed>
O	ice cubes
S	in	 [glass]
O	glass
S	contains	{ice cubes}

Pour vodka over the ice in the glass
Creating functional unit for: Pour vodka over the ice in the glass
Objects: ['vodka', 'glass']
Current object is : vodka
	 vodka state changes: {'Precondition': ['in bottle'], 'Effect': ['in glass']}
		 Precondition : in bottle || related objects: bottle
		 Effect : in glass || related objects: g

### Writing FOON to a Text File

In [6]:
# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('preprocess/'):
	os.makedirs('preprocess/')

if not os.path.exists('postprocess/'):
	os.makedirs('postprocess/')

file_ = open('preprocess/prototype.txt', 'w')
file_.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
	file_.write(unit.getFunctionalUnitText())
	unit.print_functions[2]()
	print('//')

file_.close()

O	ice cubes
S	separate
O	glass
S	empty
M	Place	<Assumed>
O	ice cubes
S	in	 [glass]
O	glass
S	contains	{ice cubes}
//
O	vodka
S	in	 [bottle]
O	glass
S	contains	{ice cubes}
M	Pour	<Assumed>
O	vodka
S	in	 [glass]
O	glass
S	contains	{ice cubes,vodka}
//
O	tomato juice
S	in	 [bottle]
O	glass
S	contains	{ice cubes,vodka}
M	Add	<Assumed>
O	tomato juice
S	in	 [glass]
O	glass
S	contains	{ice cubes,vodka,tomato juice}
//
O	lemon
S	whole
O	glass
S	contains	{ice cubes,vodka,tomato juice}
M	Squeeze	<Assumed>
O	lemon
S	squeezed
O	glass
S	contains	{ice cubes,vodka,tomato juice,lemon}
//
O	Worcestershire sauce
S	in	 [bottle]
O	glass
S	contains	{ice cubes,vodka,tomato juice,lemon}
M	Add	<Assumed>
O	Worcestershire sauce
S	in	 [glass]
O	glass
S	contains	{ice cubes,vodka,tomato juice,lemon,Worcestershire sauce}
//
O	Tabasco sauce
S	in	 [bottle]
O	glass
S	contains	{ice cubes,vodka,tomato juice,lemon,Worcestershire sauce}
M	Add	<Assumed>
O	Tabasco sauce
S	in	 [glass]
O	glass
S	contains	{ice cubes,vodka,toma

## Step 2 :- FOON to PDDL
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. (Optional) Visualize FOON graph

3. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### Parse and clean generated FOON

In [7]:
# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'	
fpa.target_dir = './postprocess/'
fpa._run_parser()

-- [FOON_parser] : Initiating parsing procedure!


 -- [FOON_parser] : Commencing parsing...
  -- parsing 'prototype.txt'...

 -- [FOON_parser] : Saving corrected files to './postprocess/'...
  -- Saving 'prototype.txt'...

-- Revising object label senses using WordNet and Concept-Net...


SUMMARY OF CHANGES:
  -- new total of OBJECTS : 16
  -- new total of STATES : 9
  -- new total of MOTIONS : 7

 -- [FOON_parser] : Parsing complete!


### (Optional) Run FOON visualization tool

In [8]:
# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
# fga._startFOONview()

### Generate PDDL files using ```FOON_to_PDDL``` module

In [9]:
FOON_subgraph_file = './postprocess/prototype.txt'

# -- definition of macro and micro plan file names:
macro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_macro.plan'
micro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_micro.plan'

# -- create a new folder for the generated problem files and their corresponding plans:
micro_problems_dir = './micro_problems-' + Path(FOON_subgraph_file).stem
if not os.path.exists(micro_problems_dir):
	os.makedirs(micro_problems_dir)

# -- perform conversion of the FOON subgraph file to PDDL:
ftp.FOON_subgraph_file = FOON_subgraph_file
ftp._convert_to_PDDL('OCP')


  -- Reading file line : 100%|██████████| 96/96 [00:00<00:00, 11982.30it/s]


 -- [FOON-fga] : Opening FOON file named './postprocess/prototype.txt'...

 -- [FOON-fga] : Building internal dictionaries...
 -- [FOON-fga] : Building output-to-FU dictionaries...
  -- Level 1: Output object map complete!
  -- Level 2: Output object map complete!
  -- Level 3: Output object map complete!

 -- [FOON-fga] : Building object-to-FU dictionaries...
  -- Level 1: Object map complete!
  -- Level 2: Object map complete!
  -- Level 3: Object map complete!

 -- [FOON_to_PDDL] : Creating domain file named './postprocess/prototype_domain.pddl'...

 -- [FOON_to_PDDL] : Creating problem file named './postprocess/prototype_problem.pddl'...
 -- [FOON_retrieval] : Creating file with kitchen items...

